In [1]:
import sys

from fedcore.api.config_factory import ConfigFactory
from torchvision.models import resnet18, ResNet18_Weights
from fedcore.api.api_configs import (
    APIConfigTemplate, AutoMLConfigTemplate, FedotConfigTemplate, 
    LearningConfigTemplate, ModelArchitectureConfigTemplate, 
    NeuralModelConfigTemplate, PruningTemplate
)
from fedcore.architecture.dataset.api_loader import ApiLoader
from fedcore.tools.example_utils import get_scenario_for_api
from fedcore.api.main import FedCore

2025-04-20 14:32:51,026 - PyTorch version 2.3.1 available.
2025-04-20 14:32:52,250 - Device <cpu> is selected


### Constants

In [2]:
METRIC_TO_OPTIMISE = ['accuracy', 'latency']
LOSS = 'norm_loss'
PROBLEM = 'classification'
PEFT_PROBLEM = 'pruning'
INITIAL_ASSUMPTION = resnet18(ResNet18_Weights)
INITIAL_MODEL = 'ResNet18'
PRETRAIN_SCENARIO = 'from_checkpoint'
SCRATCH = 'from_scratch'

POP_SIZE = 2

### Config Setup

In [3]:
initial_assumption, learning_strategy = get_scenario_for_api(
    scenario_type=PRETRAIN_SCENARIO,
    initial_assumption=INITIAL_ASSUMPTION
)

model_config = ModelArchitectureConfigTemplate(
    input_dim=None,
    output_dim=None,
    depth=6
)

pretrain_config = NeuralModelConfigTemplate(
    epochs=200,
    log_each=10,
    eval_each=15,
    save_each=50,
    criterion=LOSS,
    model_architecture=model_config,
    custom_learning_params=dict(
        use_early_stopping={
            'patience': 30,
            'maximise_task': False,
            'delta': 0.01
        }
    )
)

fedot_config = FedotConfigTemplate(
    problem=PROBLEM,
    metric=METRIC_TO_OPTIMISE,
    pop_size=POP_SIZE,
    timeout=5,
    initial_assumption=INITIAL_ASSUMPTION
)

automl_config = AutoMLConfigTemplate(fedot_config=fedot_config)

finetune_config = NeuralModelConfigTemplate(
    epochs=3,
    log_each=3,
    eval_each=3,
    criterion=LOSS,
)

peft_config = PruningTemplate(
    importance="magnitude",
    pruning_ratio=0.8,
    finetune_params=finetune_config
)

learning_config = LearningConfigTemplate(
    criterion=LOSS,
    learning_strategy=learning_strategy,
    learning_strategy_params=pretrain_config,
    peft_strategy=PEFT_PROBLEM,
    peft_strategy_params=peft_config
)

### FedCore Usage

Initialize config:

In [4]:
api_template = APIConfigTemplate(
    automl_config=automl_config,
    learning_config=learning_config
)

In [5]:
APIConfig = ConfigFactory.from_template(api_template)
api_config = APIConfig()

Load data from constant repository and convert it to FedCore format:

In [6]:
al = ApiLoader('CIFAR10', {'split_ratio': [0.6, 0.4]})
input_data = al._convert_to_fedcore(al._init_pretrain_dataset(al.source))

Files already downloaded and verified


Model compression:

In [7]:
fedcore_compressor = FedCore(api_config)
fedcore_compressor.fit(input_data)

Creating Dask Server
2025-04-20 14:32:54,133 - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2025-04-20 14:32:54,172 - State start
2025-04-20 14:32:54,180 -   Scheduler at: inproc://10.64.4.244/18600/1
2025-04-20 14:32:54,181 -   dashboard at:  http://10.64.4.244:8787/status
2025-04-20 14:32:54,193 -       Start worker at: inproc://10.64.4.244/18600/4
2025-04-20 14:32:54,194 -          Listening to:          inproc10.64.4.244
2025-04-20 14:32:54,194 -           Worker name:                          0
2025-04-20 14:32:54,195 -          dashboard at:          10.64.4.244:62327
2025-04-20 14:32:54,196 - Waiting to connect to: inproc://10.64.4.244/18600/1
2025-04-20 14:32:54,196 - -------------------------------------------------
2025-04-20 14:32:54,197 -               Threads:                          4
2025-04-20 14:32:54,197 -                Memory:                   3.98 GiB
2025-04-20 14:32:54,198 -       Loc

Generations:   0%|          | 0/10000 [00:00<?, ?gen/s]



>>> CATCHED MUTATION:
['importance', 'importance_normalize', 'pruning_iterations', 'pruning_ratio', 'importance_norm']

<<<<
=======================Prepare original model for pruning=======================
=======================Prepare original model for pruning=======================
================= Initialisation of meta_pruner pruning agent ==================
====================== Pruning importance - group_taylor =======================
====================== Pruning ratio - 0.7071428571428571 ======================
========================= Pruning importance norm -  0 =========================
================= Initialisation of meta_pruner pruning agent ==================
======================== Pruning importance - magnitude ========================
============================= Pruning ratio - 0.8 ==============================
========================= Pruning importance norm -  1 =========================
Gradients accumulation
==============After pruning=============

Batch #: 100%|██████████| 3750/3750 [02:51<00:00, 21.85it/s]


==============After pruning=================
Params: 11.689512 M => 0.989432 M
MACs: 0.037689 G => 0.003623 G
==============Finetune pruned model=================


...

Generations:   0%|          | 0/10000 [10:41<?, ?gen/s]


2025-04-20 14:43:36,591 - ApiComposer - Model generation finished
=======================Prepare original model for pruning=======================
================= Initialisation of meta_pruner pruning agent ==================
======================== Pruning importance - magnitude ========================
============================= Pruning ratio - 0.8 ==============================
========================= Pruning importance norm -  1 =========================
==============After pruning=================
Params: 11.689512 M => 0.465125 M
MACs: 0.037689 G => 0.001818 G
==============Finetune pruned model=================
Triggered OptimizerGen at 1 epoch.


Batch #: 100%|██████████| 3750/3750 [01:08<00:00, 54.44it/s]

Triggered FitReport at 3 epoch.
Train # epoch: 3, value: 4.340942634351383e-18
2025-04-20 14:47:08,908 - FEDOT logger - Final pipeline was fitted
2025-04-20 14:47:08,910 - FEDOT logger - Final pipeline: {'depth': 1, 'length': 1, 'nodes': [pruning_model]}
pruning_model - {'log_each': None, 'eval_each': None, 'save_each': None, 'epochs': 15, 'optimizer': 'adam', 'scheduler': None, 'criterion': 'norm_loss', 'custom_learning_params': {}, 'custom_criterions': {}, 'model_architecture': {'input_dim': None, 'output_dim': None, 'depth': 3, 'custom_model_params': {}}, 'importance': 'magnitude', 'importance_norm': 1, 'pruning_ratio': 0.8, 'importance_reduction': 'max', 'importance_normalize': 'max', 'pruning_iterations': 1, 'finetune_params': {'log_each': 3, 'eval_each': 3, 'save_each': None, 'epochs': 3, 'optimizer': 'adam', 'scheduler': None, 'criterion': 'norm_loss', 'custom_learning_params': {}, 'custom_criterions': {}, 'model_architecture': {'input_dim': None, 'output_dim': None, 'depth': 3,

{'depth': 1, 'length': 1, 'nodes': [pruning_model]}